# This is a multiagent simulation

This simulation is inspired by Woolley et al (2010), [Evidence for a Collective Intelligence Factor in the Performance of Human Groups](https://www.science.org/doi/full/10.1126/science.1193147). This paper demonstrates that groups with more equitable turn-taking in brainstroming and other collective tasks reach better outcomes.

In this simulation, 3 agents will be conversing with one another in a random order. They are brainstorming an idea for an ad campaign for a sustainable fashion brand which produces clothes from recycled materials. These clothes tend to be on the expensive side, but they are also more durable than conventional fashion brands. Overall, these clothes cost more, last longer, and are better for the environemnt.

### Import Libraries
This section imports libraries and provided the OpenAI key for calling the GPT4 API

In [ ]:
#!pip install autogen~=0.2

import random

try:
    from autogen import AssistantAgent, GroupChatManager, GroupChat
except ModuleNotFoundError as e:
    raise ImportError("The 'autogen' module is not installed. Please install it using 'pip install pyautogen'.") from e

llm_config = {
    "model": "gpt-5-mini",
    "api_key": "use-your-API-key",
    "base_url": "https://api.openai.com/v1"
}

### Global Parameters
This section decribes the team's task, and seeds the three simulated agents with personalities. Feel free to edit these perosnalities.

In [ ]:
# Define the task prompt
TASK_PROMPT = (
    "You are part of a team brainstorming a theme for an ad campaign for a sustainable fashion brand. "
    "This brand produces clothing made from recycled materials. The clothing is expensive but highly durable. "
    "The target audience is college-educated professionals, ages 25–45, in high-income, knowledge-based jobs."
    "We need to come up with a concept for the ad campaign."
    "You will be debating this with 2 other team members. Be constructive in your comments, and to the point."
)

# Define agent personalities
personalities = [
    "Creative marketer who loves bold visuals and emotional storytelling.",
    "Data-driven strategist focused on demographic targeting and ROI.",
    "Eco-conscious advocate passionate about authentic sustainability messaging.",
]



### Initialize Agent States
This section initializes the each agents with their personality and by describing the task.

In [ ]:
# Define specialized agents

team_agents = []
for i, personality in enumerate(personalities):

    agent = AssistantAgent(
        name=f"Member_{i+1}",
        system_message=f"You are a {personality}. {TASK_PROMPT}.",
        llm_config=llm_config,
        code_execution_config={"use_docker": False}
    )

    team_agents.append(agent)


### Speaking Weights
This parameter assigns each agent a speakig weight. Each agent's likelihood of speaking is proportional to these weights. If the weights are equal, each agent has the same likelihood to speak. If one agnet has a substantially greater weight than the other, this agent will dominate the conversation. You can edit this to explore the effects of different speaking weight distributions.

In [ ]:
# Define speaking weights (e.g., the higher the relative weight for a speaker, the more they dominate)
speaking_weights = [0.4, 0.2, 0.15]


### Define Simulation Parameters
The chat manager oversees the flow of the conversation but does not provide additional instructions to the agents.

In [ ]:
# Setup group chat object
group_chat = GroupChat(agents=team_agents, messages=[], speaker_selection_method="manual")
manager_config = {"groupchat": group_chat, "llm_config": llm_config}
chat_manager = GroupChatManager(name="manager", **manager_config)



### Simulation
This section runs the simulation.
A few things to note:
1. NUM_ROUNDS defines how many rounds the simulation will last. At the last round, the last speaking agent will be prompted to provide the group's final description.
2. An agent's likelihood of spekaing next is proportional to their weight as defined above.
3. An agent cannot speak in two rounds consecutively.

In [ ]:
# Simulate turn-based speaking with controlled order

# this parameter defines how many rounds of conversation are simulated
NUM_ROUNDS = 15

previous_speaker = None
next_speaker = None

for round in range(NUM_ROUNDS):
    # Example: random speaker selection

    while next_speaker==previous_speaker:
        next_speaker = random.choices(team_agents, weights=speaking_weights, k=1)[0]

    print(f"We are in round {round+1}, the next speaker is: {next_speaker.name}\n\n")

    suffix = ""

    # handoff instructions
    if round==0:
        msg = f"Please share your idea for the ad campaign. Make it short, no longer than two paragraphs."
    elif round==NUM_ROUNDS-1:
        msg = f"The previous speaker said: {reply}. Our conversation is about to end. Please provide the final idea for the ad campaign."
    else:
        msg = f"The previous speaker said: {reply}. Please react and share your ideas. Make it short, no longer than 2 paragraphs"

    reply = next_speaker.generate_reply(
                messages=[{"role": "user", "content": msg}],
                sender=chat_manager
            )
    print(f"🗣️ {next_speaker.name}: {reply}\n-------------------------------------------------------\n\n")

    previous_speaker = next_speaker


We are in round 1, the next speaker is: Member_1


🗣️ Member_1: Campaign: Own the Decade. A cinematic spot follows one signature piece—a trench, blazer, or parka—through 3,650 days of a professional’s life: promotions, red-eye flights, rainy bike commutes, a TED-style talk, a kid’s first recital, a climate rally. The garment ages beautifully, not breaks. On screen: “66 bottles. 10 years. 1 decision.” Tagline: Made from yesterday’s waste. Built for your next decade.

Support with premium proof points and smart tools: close-up textures, durability tests, and an Impact Receipt showing materials reclaimed, CO2 saved, and projected cost-per-wear. Interactive ads let users spin the garment across seasons and scenarios, ending with the line: Invest in what lasts—own fewer, own better, own the decade.
-------------------------------------------------------


We are in round 2, the next speaker is: Member_3


🗣️ Member_3: Love “Own the Decade.” It’s aspirational without moralizing, and the 3,65